In [1]:
import os

In [2]:
%pwd

'/config/workspace/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/config/workspace'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str

In [12]:
from MlProject.utils.common import read_yaml, create_directories
from MlProject.constants import *

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column = schema.name
        )

        return model_evaluation_config

In [18]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import numpy as np
import joblib
from MlProject.utils.common import save_json

In [20]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual_val, predicted_val):
        rmse =np.sqrt(mean_squared_error(actual_val, predicted_val))
        mae = mean_absolute_error(actual_val, predicted_val)
        r2 = r2_score(actual_val, predicted_val)

        return {"rmse":rmse, "mae":mae, "r2":r2}

    def save_results(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        x_test =test_data.drop([self.config.target_column],axis=1)
        y_test = test_data[[self.config.target_column]]

        predicted_val = model.predict(x_test)
        scores = self.eval_metrics(y_test, predicted_val)

        save_json(path=Path(self.config.metric_file_name), data=scores)

In [21]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.save_results()
except Exception as e:
    raise(e)

[2024-01-08 15:28:08,326: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-08 15:28:08,329: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-08 15:28:08,333: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-08 15:28:08,334: INFO: common: created directory at: artifacts]
[2024-01-08 15:28:08,336: INFO: common: created directory at: artifacts/model_evaluation]
[2024-01-08 15:28:08,348: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]
